In [ ]:
import numpy as np

from ptable_dict import ptable, atomic_masses
from utilities import write_xyz, load_xyz

# calculate vectors for large MD box

In [ ]:

# 147.2122 x 137.2347 x 102.7109 (abc)
# 85.00009, 78.38998, 59.10033 (alpha, beta, gamma)
def calc_real_space_abc(a_mag, b_mag, c_mag, alpha_deg, beta_deg, gamma_deg):
    '''
    https://www.ucl.ac.uk/~rmhajc0/frorth.pdf
    '''
    alpha = np.deg2rad(alpha_deg)
    beta = np.deg2rad(beta_deg)
    gamma = np.deg2rad(gamma_deg)
    
    V = a_mag*b_mag*c_mag*np.sqrt(1-np.cos(alpha)**2-np.cos(beta)**2-np.cos(gamma)**2+2*np.cos(alpha)*np.cos(beta)*np.cos(gamma)) 
    
    ax = a_mag
    ay = 0
    az = 0
    a = np.array([ax, ay, az])
    
    bx = b_mag*np.cos(gamma)
    by = b_mag*np.sin(gamma)
    bz = 0
    b = np.array([bx, by, bz])
    
    cx = c_mag*np.cos(beta)
    cy = c_mag*(np.cos(alpha)-np.cos(beta)*np.cos(gamma))/(np.sin(gamma))
    cz = V/(a_mag*b_mag*np.sin(gamma))
    c = np.array([cx, cy, cz])
    
    return a, b, c

calc_real_space_abc(147.212, 137.235, 102.711, 85.00009, 78.38998, 59.10033)

# Use vectors calculated above to repeat MD box

In [ ]:
dirr = '/Users/Thomas2/Library/CloudStorage/OneDrive-UCB-O365/Desktop/Research_Stuff/OPV_GIWAXS/giwaxs_forward_sim/test_xyz_files/'
xyz_file = f'{dirr}rectangular_N2200_test2.xyz'
coords, elements = load_xyz(xyz_file)

x_max = np.max(coords[:,0])-np.min(coords[:,0])
y_max = np.max(coords[:,1])-np.min(coords[:,1])
z_max = np.max(coords[:,2])-np.min(coords[:,2])

print(x_max)
print(y_max)
print(z_max)

In [ ]:
dirr = '/Users/Thomas2/Library/CloudStorage/OneDrive-UCB-O365/Desktop/Research_Stuff/OPV_GIWAXS/N2200_simulation/dump_files/'
names = ['dump_10', 'dump_20', 'dump_30', 'dump_40', 'dump_50', 'dump_60', 'dump_70', 'dump_80', 'dump_90', 'dump_100']

for name in names:
    xyz_file = f'{dirr}{name}.xyz'
    coords, elements = load_xyz(xyz_file)
    
    # number of adds in each direction
    num_x = 2
    num_y = 0
    num_z = 2
    
    x_max = np.max(coords[:,0])-np.min(coords[:,0])
    y_max = np.max(coords[:,1])-np.min(coords[:,1])
    z_max = np.max(coords[:,2])-np.min(coords[:,2])
    
    coords_original = coords
    elements_original = elements
    for i in range(num_x):
        # coords_new = coords_original+[x_max*(i+1), 0, 0]
        coords_new = coords_original+[147.212*(i+1), 0, 0]
        if i == 0:
            coords_append = coords_new
            elements_append = elements_original
        else:
            coords_append = np.concatenate((coords_append, coords_new), axis=0)
            elements_append = np.concatenate((elements_append, elements), axis=0)  
        if i==num_x-1:
            coords = np.concatenate((coords_original, coords_append), axis=0)
            elements = np.concatenate((elements_original, elements_append), axis=0)
    
    coords_original = coords
    elements_original = elements
    for i in range(num_y):
        # coords_new = coords_original+[0, y_max*(i+1), 0]
        coords_new = coords_original+[70.4751555*(i+1), 117.75694325*(i+1), 0]
        if i ==0:
            coords_append = coords_new
            elements_append = elements_original
        else:
            coords_append = np.concatenate((coords_append, coords_new), axis=0)
            elements_append = np.concatenate((elements_append, elements), axis=0)
        if i==num_y-1:
            coords = np.concatenate((coords_original, coords_append), axis=0)
            elements = np.concatenate((elements_original, elements_append), axis=0)
    
    coords_original = coords
    elements_original = elements
    for i in range(num_z):
        # coords_new = coords_original+[0, 0, z_max*(i+1)]
        # coords_new = coords_original + [39.33725991*(i+1), -6.33507644*(i+1), 65.42176083*(i+1)]
        coords_new = coords_original + [20.67050947*(i+1),  -1.93849981*(i+1), 100.59086329*(i+1)]
        if i ==0:
            coords_append = coords_new
            elements_append = elements_original
        else:
            coords_append = np.concatenate((coords_append, coords_new), axis=0)
            elements_append = np.concatenate((elements_append, elements), axis=0)
        if i==num_z-1:
            coords = np.concatenate((coords_original, coords_append), axis=0)
            elements = np.concatenate((elements_original, elements_append), axis=0)
    
    save_path = f'{dirr}{name}_{num_x+1}x{num_y+1}x{num_z+1}.xyz'
    write_xyz(save_path, coords, elements)

    del coords, coords_append, coords_new, coords_original
    del elements, elements_append, elements_original

# slice box into rectangular prism

In [ ]:
#make rectangular prism
dirr = '/Users/Thomas2/Library/CloudStorage/OneDrive-UCB-O365/Desktop/Research_Stuff/OPV_GIWAXS/N2200_simulation/dump_files/'
names = ['dump_10', 'dump_20', 'dump_30', 'dump_40', 'dump_50', 'dump_60', 'dump_70', 'dump_80', 'dump_90', 'dump_100']
for name in names:
    xyz_file = f'{dirr}{name}_3x1x3.xyz'
    coords, elements = load_xyz(xyz_file)
    
    x_max = np.max(coords[:,0])-np.min(coords[:,0])
    y_max = np.max(coords[:,1])-np.min(coords[:,1])
    z_max = np.max(coords[:,2])-np.min(coords[:,2])
    
    x_buffer = 125
    y_buffer = 25
    z_buffer = 10
    
    x_lower = x_buffer
    x_upper = x_max-x_buffer
    y_lower = y_buffer
    y_upper = y_max-y_buffer
    z_lower = z_buffer
    z_upper = z_max-z_buffer
    
    # Shift coords array to origin 
    coords[:,0] -= np.min(coords[:,0])
    coords[:,1] -= np.min(coords[:,1])
    coords[:,2] -= np.min(coords[:,2])
    
    # Use NumPy masking to filter the coordinates
    mask = (
        (coords[:,0] >= x_lower) & (coords[:,0] <= x_upper) &
        (coords[:,1] >= y_lower) & (coords[:,1] <= y_upper) &
        (coords[:,2] >= z_lower) & (coords[:,2] <= z_upper)
    )
    
    coords_new = coords[mask]
    elements_new = elements[mask]
    
    
    save_path = f'{dirr}{name}_rect_cut125x25x10.xyz'
    write_xyz(save_path, coords_new, elements_new)

# Sum multiple .xyz files

In [ ]:
dirr = '/Users/Thomas2/Library/CloudStorage/OneDrive-UCB-O365/Desktop/Research_Stuff/OPV_GIWAXS/N2200_simulation/dump_files/'
names = ['dump_10', 'dump_20', 'dump_30', 'dump_40', 'dump_50', 'dump_60', 'dump_70', 'dump_80', 'dump_90', 'dump_100']

for i, name in enumerate(names):
    xyz_file = f'{dirr}{name}_rect_cut125x25x10.xyz'
    coords, elements = load_xyz(xyz_file)
    if i==0:
        coords_append = coords
        elements_append = elements
    else:
        coords_append = np.concatenate((coords_append, coords), axis=0)
        elements_append = np.concatenate((elements_append, elements), axis=0)

save_path = f'{dirr}sum_rect_cut125x25x10.xyz'
write_xyz(save_path, coords_append, elements_append)